# Make sure to run each cell in order!

In [ ]:
#install required info
pip install google-api-python-client
pip install gspread oauth2client

Google Calendar API setup

In [2]:
#importing libraries
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from pprint import pprint
from datetime import datetime, timedelta
import pickle

In [372]:
scopes = ['https://www.googleapis.com/auth/calendar'] 

In [374]:
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)

In [375]:
credentials = flow.run_console()   #only run once this when you want to give access to your calendar

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=986237573878-m3chvhga4rk6lr1efspk3e4gtji6kuf1.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=3iiPJFPjHRoGHlpD2ppIg2Vls2lg1k&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWiQPZknHZzYfkrTsMW1JR3tRIPqbkthGcHOkPiw7D_cIACeq_THh8U


In [376]:
import pickle #lets save the credentials into a specific file!

#so we don't have to run the previous cells each time

In [377]:
pickle.dump(credentials, open("token.pkl", "wb")) 
#dump is used for dumping any object into a file?

In [4]:
credentials = pickle.load(open("token.pkl","rb"))
#opens the credentials 
#which we saved into this

In [5]:
credentials #here are the credentials with the authorization code

# Get calendar info

In [6]:
service = build("calendar","v3", credentials=credentials) #builds calendar info

calendar_list = service.calendarList().list(pageToken=page_token).execute()

In [16]:
result = service.calendarList().list().execute()
result #returns a list of all of the calendars you are involved in ( NOT EVENTS, LITERAL CALENDARS)

result['items'][0] #first calendar in calendar list

#we now have the calendar linked to the email, look carefully at the "id" tag and you'll see


{'kind': 'calendar#calendarListEntry',
 'etag': '"1593136794934000"',
 'id': 'peircesherpa@gmail.com',
 'summary': 'peircesherpa@gmail.com',
 'timeZone': 'America/Los_Angeles',
 'colorId': '14',
 'backgroundColor': '#9fe1e7',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 30}],
 'notificationSettings': {'notifications': [{'type': 'eventCreation',
    'method': 'email'},
   {'type': 'eventChange', 'method': 'email'},
   {'type': 'eventCancellation', 'method': 'email'},
   {'type': 'eventResponse', 'method': 'email'}]},
 'primary': True,
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

# Lets Get The Events Within a calendar!

In [18]:
#RUN THE CELL BEFORE !!!

calendar_id = result['items'][0]['id']
print(calendar_id)

result = service.events().list(calendarId=calendar_id).execute() #makes a list of all calendar events in calendar

result['items']
#makes a dictionary holding all events under gmail calendar id, and their properties.

peircesherpa@gmail.com


[{'kind': 'calendar#event',
  'etag': '"3200571560834000"',
  'id': 'su4kgh4skpdcb1k2nqj49c6hms',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=c3U0a2doNHNrcGRjYjFrMm5xajQ5YzZobXMgcGVpcmNlc2hlcnBhQG0',
  'created': '2020-09-16T19:38:25.000Z',
  'updated': '2020-09-16T19:49:40.417Z',
  'summary': 'Respond to presidential elections flip grid at least 2',
  'colorId': '6',
  'creator': {'email': 'peircesherpa@gmail.com', 'self': True},
  'organizer': {'email': 'peircesherpa@gmail.com', 'self': True},
  'start': {'date': '2020-09-16'},
  'end': {'date': '2020-09-17'},
  'iCalUID': 'su4kgh4skpdcb1k2nqj49c6hms@google.com',
  'sequence': 0,
  'reminders': {'useDefault': False},
  'eventType': 'default'},
 {'kind': 'calendar#event',
  'etag': '"3200572633018000"',
  'id': 'eomcbp18rtvgi26ra6er5co260',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=ZW9tY2JwMThydHZnaTI2cmE2ZXI1Y28yNjAgcGVpcmNlc2hlcnBhQG0',
  'created': '2020-

# Get My Calendar Events

# Google Sheets API setup

In [22]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials #you'll need a google developers account for this
from pprint import pprint #pprint = "pretty print"

scopes = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive","https://www.googleapis.com/auth/calendar"] #I might need to use: #https://www.googleapis.com/auth/drive
creds = ServiceAccountCredentials.from_json_keyfile_name("PeirceService.json", scopes) #you'll need to get service account credentials and input the json file name before scopes
client = gspread.authorize(creds)
sheet = client.open("Automation Trial 1").sheet1 #name of the sheet that you're trying to open

data = sheet.get_all_records()
pprint(data)

[]


How to insert rows:
insertRow = ['hello', 5, 'red','blue'] 
sheet.insert_row(insertRow,4)
.insert_row(object_of_things_you_wanna_insert, row_number)

In [24]:
#lets make it insert rows of calendar data
import datetime

startDate = input('enter a start date in the format mm-dd-yyyy')
endDate = input('enter a start date in the format mm-dd-yyyy')
start = datetime.datetime.strptime(startDate, "%m-%d-%Y") #converting it to datetime
end = datetime.datetime.strptime(endDate, "%m-%d-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, ((int((end-start).days))+1))] #finds date range
dateslist = []
for date in date_generated:
    dateslist.append(date.strftime("%Y-%m-%d")) #creates list of dates in date range

#this try statement is just to make sure all events that are read have a dateTime thing
rowNumber = 2 #we are starting at 2nd row because the first row has desciptions in the sheet
for date in result['items']:
    try: #only uses events with dateTime tag, this eliminates events without duration in hours
        if date['start']['dateTime'].split('T')[0] in dateslist:  #finds the start dates that are in the dates list
            try: #only inserts rows that have all criteria. MAINLY DESCRIPTION
                description = (date['description'])
                summary = (date['summary'])
                startTime = (str(date['start']['dateTime'].split('T')[1]))
                endTime = (str(date['end']['dateTime'].split('T')[1]))
                eventday = (str(date['start']['dateTime'].split('T')[0]))
                print(summary, description, startTime, endTime, eventday + "\n") #printing information that's going to be on sheet
                newRow = (eventday, summary, description, startTime, endTime) #This is the line of code going into the sheet
                
                sheet.insert_row(newRow, rowNumber) #adds line of code to add row to sheet
                
                rowNumber+=1 #moves to next row to add information into it
                
            except KeyError: 
                #the sucessful values are printed and added to the sheet
                #the except: key error values only get printed
                description = '\n Description: none' #for calendar values without description
                summary = ('Summary: ' + date['summary'])
                startTime = ('\nStart Time: '+ str(date['start']['dateTime'].split('T')[1]))
                endTime = ('\nEnd Time: '+ str(date['end']['dateTime'].split('T')[1]))
                eventday = ('\nDate: '+ str(date['start']['dateTime'].split('T')[0]))
                print(summary, description, startTime, endTime, eventday + "\n")
    except:
        pass



enter a start date in the format mm-dd-yyyy06-06-2022
enter a start date in the format mm-dd-yyyy06-10-2022
Work Used a microwave and some grapes to make plasma today.
Definitely did a lot of work! 09:00:00-07:00 16:30:00-07:00 2022-06-06

Work Again Sold immense amounts of popcorn in the stands. 09:00:00-07:00 17:00:00-07:00 2022-06-07

School I am now a smarter human 09:00:00-07:00 15:30:00-07:00 2022-06-08

Skool All day every dayyy lets get this!
Okay but really lets get this CS project finished 09:00:00-07:00 16:45:00-07:00 2022-06-09

Work Printed out designs for clients, worked on fixing website design flaws 09:00:00-07:00 16:15:00-07:00 2022-06-10

